In [2]:
scope.dis()
target.dis()

In [5]:
exit()

In [1]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
# scope.adc.samples=24400

In [2]:
print(scope.clock.adc_src)
scope.clock.adc_src = "clkgen_x4"

clkgen_x4


In [12]:
def join_rand(*args):
    args = list(args)
    random.shuffle(args)
    return reduce(lambda x,y: x+y, args)

print(join_rand([0],[1],[0]))

[0, 1, 0]


In [ ]:
# 3rd order masking
# function
def setup():
    # runs once in the start -> prints returned
    load(scope, "../c-files/xor-CWLITEARM")
    print(date_str())
    pass
    
def execute(index, op, t):
    if op == "set":
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = join_rand(xor(x,x_t1,x_t2), x_t1, x_t2) + join_rand(xor(y,y_t1,y_t2), y_t1, y_t2) + [0 for _ in range(env.read*8-size*6)]
    else:
        x_ = [random.randrange(2) for _ in range(size)]
        y_ = [random.randrange(2) for _ in range(size)]
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = join_rand(xor(x_,x_t1,x_t2), x_t1, x_t2) + join_rand(xor(y_,y_t1,y_t2), y_t1, y_t2) + [0 for _ in range(env.read*8-size*6)]
    return get_bytes(msg)

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    # print(xor(get_bits(fdb)[:size], get_bits(fdb)[size:2*size]))
    if not t:
        samps = scope.adc.trig_count
        print(samps)
    
    if op == "set":
        if xor(x,y) != xor(get_bits(fdb)[:size], get_bits(fdb)[size:2*size], get_bits(fdb)[2*size:3*size]):
            print("NO!")
    if fdb is None:
        return
    
size = 32
env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set", "rand"] # operations list
env.subfolder = "xor" # subfolder name
env.TRACES_NUM = 50_000 # number of traces
scope.adc.samples = 64 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.index = 0
env.read = 12
env.desc = "the XOR ordering is randomized each run"
env.title = "3rd order masking, 200K total samples, set vs rand"
env.name = "xor_ord3_100K_rand-switch"
s = Scheme() # scheme execution helper

x = [random.randrange(2) for _ in range(size)]
y = [random.randrange(2) for _ in range(size)]


# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

status = traces("xor", scope, target, env)

generate_file(env, globals(), "start_time", "operations", "x", "y")
print("done", date_str())
# TODO: Check combine file functionality

In [5]:
------ STASH --------- DO NOT RUN

# normal order 3 masking
def execute(index, op, t):
    if op == "set":
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = xor(x,x_t1,x_t2) + x_t1 + x_t2 + xor(y,y_t1,y_t2) + y_t1 + y_t2 + [0 for _ in range(env.read*8-size*6)]
    else:
        x_ = [random.randrange(2) for _ in range(size)]
        y_ = [random.randrange(2) for _ in range(size)]
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = xor(x_,x_t1,x_t2) + x_t1 + x_t2 + xor(y_,y_t1,y_t2) + y_t1 + y_t2 + [0 for _ in range(env.read*8-size*6)]
    return get_bytes(msg)

def execute(index, op, t):
    if op == "set":
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = xor(x,x_t1,x_t2) + x_t1 + x_t2 + y_t2 + xor(y,y_t1,y_t2) + y_t1 + [0 for _ in range(env.read*8-size*6)]
    else:
        x_ = [random.randrange(2) for _ in range(size)]
        y_ = [random.randrange(2) for _ in range(size)]
        x_t1 = [random.randrange(2) for _ in range(size)]
        y_t1 = [random.randrange(2) for _ in range(size)]
        x_t2 = [random.randrange(2) for _ in range(size)]
        y_t2 = [random.randrange(2) for _ in range(size)]
        msg = xor(x_,x_t1,x_t2) + x_t1 + x_t2 + y_t1 + xor(y_,y_t1,y_t2) + y_t2 + [0 for _ in range(env.read*8-size*6)]
    return get_bytes(msg)

[0, 1, 1, 1, 0]

In [18]:
# 2nd order masking
# function
def setup():
    # runs once in the start -> prints returned
    load(scope, "../c-files/xor2-CWLITEARM")
    print(date_str())
    pass
    
def execute(index, op, t):
    global tc
    tc = scope.adc.trig_count
    if op == "set":
        x_t = [random.randrange(2) for _ in range(size)]
        y_t = [random.randrange(2) for _ in range(size)]
        msg = xor(x,x_t) + x_t + xor(y,y_t) + y_t + [0 for _ in range(128-size*4)]
    else:
        x_ = [random.randrange(2) for _ in range(size)]
        y_ = [random.randrange(2) for _ in range(size)]
        x_t = [random.randrange(2) for _ in range(size)]
        y_t = [random.randrange(2) for _ in range(size)]
        msg = xor(x_,x_t) + x_t + xor(y_,y_t) + y_t + [0 for _ in range(128-size*4)]
    return get_bytes(msg)

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    # print(xor(get_bits(fdb)[:size], get_bits(fdb)[size:2*size]))
    if not t:
        samps = scope.adc.trig_count
        print(samps)
    if fdb is None:
        return
    if op == "set":
        if xor(x,y) != xor(get_bits(fdb)[:size], get_bits(fdb)[size:2*size]):
            print("NO!")
    
size = 32
env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set", "rand"] # operations list
env.subfolder = "xor" # subfolder name
env.TRACES_NUM = 50_000 # number of traces
scope.adc.samples = 60 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = "set - xor between two initially random variables"
env.title = "2nd order masking, 50k samples, set vs rand"
env.name = "xor_ord2_50K_repeat"
s = Scheme() # scheme execution helper

tc = 0

x = [random.randrange(2) for _ in range(size)]
y = [1-random.randrange(2) for _ in range(size)]


# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

status = traces("xor", scope, target, env)

generate_file(env, globals(), "start_time", "operations")
print("done", date_str())

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4939 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4939 bytes
2024-06-04 13-19
None
60
60
xor done 2024-06-04 14-12


In [ ]:
# masked random KEY
# function
def setup():
    # runs once in the start -> prints returned
    # load(scope, "../c-files/masked-CWLITEARM")
    print(date_str())
    pass
    
def execute(index, op, t):
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    global rand
    if op in ["random", "tvla"]:
        k_temp = [random.randrange(2) for _ in range(64)]
    elif op == "set":
        k_temp = k1
    elif op == "zero":
        k_temp = k0
    else:
        return None
    rand1 = [random.randrange(2) for _ in range(64)]
    rand = [random.randrange(1) for _ in range(64)]
    mem = xor(puf, s.encode(k))
    posture(target,'k', get_bytes(xor(mem, s.encode(rand1))), 16)
    p_temp = xor(puf, s.encode(rand)) # , mem, s.encode(rand)
    rand = xor(rand,rand1)
    # time.sleep(0.005)
    return get_bytes(p_temp)

def ending(fdb, op, t):
    # runs after each CW command -> True <=> trace isnt taken
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (xor(get_bits(fdb)[:64], rand) != k):
        print("WRONG!")
        return True
    pass


env = ENV()
# operation settings
env.root = "F:/programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["random", "tvla", "set", "zero"] # operations list
env.subfolder = "masked" # subfolder name
env.TRACES_NUM = 1250 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.title = ""

env.desc = "PUF's mask is not refreshed"
prefix_name = "okey_masked_decode"
env.name = prefix_name
env.readme = """masked decoding
TVLA between set key + puf against random key + set PUF
additional randomized key vs randomized key for sanity
public memory is masked, but PUF isnt"""

s = Scheme() # scheme execution helper


rand = None

k0 = [0 for _ in range(64)] #PUF([random.randrange(2) for _ in range(127)])
k1 = [random.randrange(2) for _ in range(64)] #PUF([random.randrange(2) for _ in range(127)])
puf = [random.randrange(2) for _ in range(127)]
set_weight = sum(p1.puf)
key_weight = sum(k)


# run loop
print(setup())
time.sleep(0.1)
start_time = date_str()

file_names = [f"file{i}" for i in range(2)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    


env.title = "Once key masked Decode TVLA random vs random"
env.name = prefix_name + "_rand"
split_file(env, file_names, [0, 1], "rand")

env.title = "Once Key masked Decode TVLA random vs set"
env.name = prefix_name + "_set"
split_file(env, file_names, [0, 2], "set")

env.title = "Once Key masked Decode TVLA random vs zero"
env.name = prefix_name + "_zero"
split_file(env, file_names, [0, 3], "zero")


remove_files(env, file_names)

##

generate_file(env, globals(), "start_time", "operations", "set_weight", "key_weight")
print("done", date_str())
